# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [22]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [23]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../WeatherPy/output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,namsos,64.4662,11.4957,21.03,72,20,4.12,NO,1723070731
1,1,lata,40.1629,-8.3327,13.61,97,1,0.75,PT,1723070733
2,2,port alfred,-33.5906,26.8910,16.81,47,4,4.44,ZA,1723070734
3,3,negapatam,10.7667,79.8333,26.88,81,100,3.61,IN,1723070735
4,4,bethel,41.3712,-73.4140,19.80,86,94,0.45,US,1723070736


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [24]:
%%capture --no-display

# Configure the map plot
all_cities_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 0.5,
    color = "City",
)


# Display the map
all_cities_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [25]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_city_data_df = city_data_df.loc[(city_data_df["Max Temp"] < 27) & (city_data_df["Max Temp"] > 21) & (city_data_df["Cloudiness"] == 0) & (city_data_df["Wind Speed"] < 5) & (city_data_df["Humidity"] <= 60)].copy()

# Drop any rows with null values
ideal_city_data_df.dropna(inplace = True)

# Display sample data
ideal_city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
138,138,corinto,37.9407,22.9573,26.61,57,0,0.85,GR,1723070901
208,208,alvorada,-12.4800,-49.1247,25.56,25,0,1.77,BR,1723070795
232,232,astrakhan,46.3497,48.0408,24.93,57,0,4.00,RU,1723071018
241,241,pul-e 'alam,33.9953,69.0227,21.18,46,0,1.50,AF,1723071028
404,404,ketchikan,55.3422,-131.6461,23.26,49,0,2.57,US,1723071227
420,420,salmas,38.1973,44.7653,22.73,56,0,2.02,IR,1723071246
423,423,nova xavantina,-14.6667,-52.3500,25.34,33,0,1.71,BR,1723071249
438,438,sao miguel do araguaia,-13.2750,-50.1628,26.09,23,0,1.28,BR,1723071269
441,441,buka,40.8108,69.1986,23.21,29,0,3.42,UZ,1723071272
482,482,kuqa,41.7278,82.9364,21.83,55,0,2.02,CN,1723071321


### Step 3: Create a new DataFrame called `hotel_df`.

In [26]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_city_data_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
138,corinto,GR,37.9407,22.9573,57,
208,alvorada,BR,-12.4800,-49.1247,25,
232,astrakhan,RU,46.3497,48.0408,57,
241,pul-e 'alam,AF,33.9953,69.0227,46,
404,ketchikan,US,55.3422,-131.6461,49,
420,salmas,IR,38.1973,44.7653,56,
423,nova xavantina,BR,-14.6667,-52.3500,33,
438,sao miguel do araguaia,BR,-13.2750,-50.1628,23,
441,buka,UZ,40.8108,69.1986,29,
482,kuqa,CN,41.7278,82.9364,55,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [27]:
radius = 10000
limit = 1
params = {"categories":"accommodation.hotel", "limit":limit, "apiKey":geoapify_key}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    longitude = row['Lng']
    latitude = row['Lat']

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params = params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
corinto - nearest hotel: Apollon
alvorada - nearest hotel: No hotel found
astrakhan - nearest hotel: Omega
pul-e 'alam - nearest hotel: No hotel found
ketchikan - nearest hotel: Cape Fox Lodge
salmas - nearest hotel: هتل آتا
nova xavantina - nearest hotel: Hotel Morada do Sol
sao miguel do araguaia - nearest hotel: Hotel São Miguel
buka - nearest hotel: No hotel found
kuqa - nearest hotel: Kuqa Hotel - foreigners accepted
napanee - nearest hotel: Hampton
bom jesus - nearest hotel: Hotel Familiar


,City,Country,Lat,Lng,Humidity,Hotel Name
138,corinto,GR,37.9407,22.9573,57,Apollon
208,alvorada,BR,-12.4800,-49.1247,25,No hotel found
232,astrakhan,RU,46.3497,48.0408,57,Omega
241,pul-e 'alam,AF,33.9953,69.0227,46,No hotel found
404,ketchikan,US,55.3422,-131.6461,49,Cape Fox Lodge
420,salmas,IR,38.1973,44.7653,56,هتل آتا
423,nova xavantina,BR,-14.6667,-52.3500,33,Hotel Morada do Sol
438,sao miguel do araguaia,BR,-13.2750,-50.1628,23,Hotel São Miguel
441,buka,UZ,40.8108,69.1986,29,No hotel found
482,kuqa,CN,41.7278,82.9364,55,Kuqa Hotel - foreigners accepted


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [28]:
%%capture --no-display

# Configure the map plot
ideal_cities_plot = hotel_df.hvplot.points("Lng", "Lat", geo = True, tiles = "OSM", frame_width = 700, frame_height = 500, size = "Humidity", scale = 1, color = "City", hover_cols = ["Hotel Name", "Country"])

# Display the map
ideal_cities_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)